In [3]:
import chromadb
import uuid
import ollama

# Configuration
MODEL_NAME = "llama3.1:8b"
DB_PATH = r"C:\Users\hari7\Documents\Anokha Hackthon\ChromaDB"

# 1. Setup Database
print(f"Initializing ChromaDB at {DB_PATH}")
client = chromadb.PersistentClient(path=DB_PATH)
collection = client.get_or_create_collection(name="user_test_facts")

# 2. Helper Functions

def save_fact(fact):
    print(f"Saving: '{fact}'")
    collection.add(
        documents=[fact],
        ids=[str(uuid.uuid4())]
    )

def query_and_answer(user_question):
    print(f"\nQuestion: {user_question}")
    
    # A. Retrieve
    results = collection.query(
        query_texts=[user_question],
        n_results=2 
    )
    retrieved_facts = results['documents'][0]
    
    print(f"Retrieved Context: {retrieved_facts}")

    if not retrieved_facts:
        print("No relevant memory found.")
        return

    # --- THE GENERALIZED PROMPT ---
    # This prompt works for ANY user and ANY context.
    prompt = f"""
    You are a helpful AI Assistant with long-term memory.
    
    =========================================
    MEMORY BLOCK (Past statements from the User):
    {retrieved_facts}
    =========================================
    
    INSTRUCTIONS:
    1. The 'MEMORY BLOCK' contains facts the user told you previously. 
    2. If the memory says "I am X" or "I like Y", it refers to the USER (the person you are talking to).
    3. Use this memory to answer the user's current question accurately.
    4. Speak naturally. Do not say "The memory says...". Just answer as if you know it.
    
    USER QUESTION: {user_question}
    
    ANSWER:
    """
    
    # C. Generate
    response = ollama.chat(model=MODEL_NAME, messages=[{'role': 'user', 'content': prompt}])
    print(f"AI Answer: {response['message']['content']}")

# 3. Test Execution

if __name__ == "__main__":
    
    # Only run the teaching phase if you want to add NEW facts. 
    # Since you already ran it once, the data is saved!
    # You can comment out the 'facts' loop if you don't want duplicate data.
    
    # facts = [
    #     "My name is Hari.",
    #     "I am an expert in Python and Flask.",
    #     "I prefer working in startups rather than MNCs.",
    #     "I live in the southern part of India."
    # ]
    # for f in facts:
    #     save_fact(f)

    print("\nPhase 2: Testing Generalization...")

    # Test 1: Identity
    query_and_answer("Who am I?")

    # Test 2: Inference
    query_and_answer("Do I know how to code?")

    # Test 3: Location
    query_and_answer("Where do I live?")

Initializing ChromaDB at C:\Users\hari7\Documents\Anokha Hackthon\ChromaDB

Phase 2: Testing Generalization...

Question: Who am I?
Retrieved Context: ['My name is Hari.', 'My name is Hari.']
AI Answer: You're Hari.

Question: Do I know how to code?
Retrieved Context: ['I am an expert in Python and Flask.', 'I am an expert in Python and Flask.']
AI Answer: You're proficient in coding, specifically Python and Flask.

Question: Where do I live?
Retrieved Context: ['I live in the southern part of India.', 'I live in the southern part of India.']
AI Answer: You live in the southern part of India.


In [5]:
import chromadb
import uuid
import ollama

# Configuration
MODEL_NAME = "llama3.1:8b"
DB_PATH = r"C:\Users\hari7\Documents\Anokha Hackthon\ChromaDB"

# Initialize Client
client = chromadb.PersistentClient(path=DB_PATH)

# Clean Start
try:
    client.delete_collection("user_test_facts")
except:
    pass

collection = client.get_or_create_collection(name="user_test_facts")

# Helper Functions

def save_fact(fact):
    print(f"Saving User Fact: '{fact}'")
    collection.add(
        documents=[fact],
        ids=[str(uuid.uuid4())]
    )

def ask_agent(question):
    print(f"\nUser asks: \"{question}\"")
    
    # Retrieve
    results = collection.query(
        query_texts=[question], 
        n_results=3 
    )
    retrieved_memory = results['documents'][0]
    
    print(f"AI Remembers: {retrieved_memory}")
    
    # Generate
    prompt = f"""
    You are a Career Counselor.
    
    MEMORY OF USER (What they told you before):
    {retrieved_memory}
    
    USER QUESTION: "{question}"
    
    INSTRUCTION:
    Answer the question based strictly on the User's MEMORY. 
    Do not give generic advice. Tailor the specific job titles to their skills found in memory.
    """
    
    response = ollama.chat(model=MODEL_NAME, messages=[{'role': 'user', 'content': prompt}])
    print(f"\nAI Reply: {response['message']['content']}")

# Execution

if __name__ == "__main__":
    print("STEP 1: Teaching the AI your Profile")
    
    save_fact("My name is Hari.")
    save_fact("I am proficient in Python and MATLAB.")
    save_fact("I have worked on projects involving Signal Processing and EEG data analysis.")
    save_fact("I am interested in the medical or healthcare technology field.")

    print("\nSTEP 2: The Real-Life Test")
    
    ask_agent("What kind of internships should I apply for?")

STEP 1: Teaching the AI your Profile
Saving User Fact: 'My name is Hari.'
Saving User Fact: 'I am proficient in Python and MATLAB.'
Saving User Fact: 'I have worked on projects involving Signal Processing and EEG data analysis.'
Saving User Fact: 'I am interested in the medical or healthcare technology field.'

STEP 2: The Real-Life Test

User asks: "What kind of internships should I apply for?"
AI Remembers: ['I am interested in the medical or healthcare technology field.', 'I am proficient in Python and MATLAB.', 'I have worked on projects involving Signal Processing and EEG data analysis.']

AI Reply: Based on your interest in medical or healthcare technology and your proficiency in Python and MATLAB, along with experience in Signal Processing and EEG data analysis, here are some internship suggestions tailored to your skills:

1. **Signal Processing Internship**: Apply for internships that involve working with signal processing algorithms to analyze biomedical signals. This could i

In [ ]:
import chromadb
import uuid
import ollama

MODEL_NAME = "llama3.1:8b"
DB_PATH = r"C:\Users\hari7\Documents\Anokha Hackthon\ChromaDB"

client = chromadb.PersistentClient(path=DB_PATH)
collection = client.get_or_create_collection(name="user_test_facts")

def auto_save_memory(user_input):
    """
    This function uses the LLM to decide IF something should be saved.
    """
    
    # 1. Ask LLM to extract facts
    extraction_prompt = f"""
    Analyze the following user text. 
    If the user mentions a fact about themselves (Name, Skills, Experience, Goals), extract it as a short sentence.
    If there is no factual info (e.g., "Hello", "Thank you"), return exactly "NO_FACT".
    
    User Text: "{user_input}"
    """
    
    response = ollama.chat(model=MODEL_NAME, messages=[{'role': 'user', 'content': extraction_prompt}])
    extracted_data = response['message']['content'].strip()
    
    # 2. Logic to Save
    if "NO_FACT" not in extracted_data:
        print(f"Auto-Extracting info: {extracted_data}")
        collection.add(
            documents=[extracted_data],
            ids=[str(uuid.uuid4())]
        )
        print("Saved to Memory automatically.")
    else:
        print("No new facts detected in this message.")

# --- Simulation ---

if __name__ == "__main__":
    
    # Scenario 1: User introduces themselves (LLM should detect this)
    user_message_1 = "Hi, I am Hari and I am learning AI."
    print(f"\nUser says: {user_message_1}")
    auto_save_memory(user_message_1)
    
    # Scenario 2: User says something random (LLM should ignore this)
    user_message_2 = "Can you tell me a joke?"
    print(f"\nUser says: {user_message_2}")
    auto_save_memory(user_message_2)
    
    # Scenario 3: User adds a skill (LLM should detect this)
    user_message_3 = "I recently started working with Docker containers."
    print(f"\nUser says: {user_message_3}")
    auto_save_memory(user_message_3)


User says: Hi, I am Hari and I am learning AI.
Auto-Extracting info: The fact extracted from the user text is:
Hari is learning AI.
Saved to Memory automatically.

User says: Can you tell me a joke?
No new facts detected in this message.

User says: I recently started working with Docker containers.
Auto-Extracting info: The extracted fact from the user text is:

"I recently started working with Docker containers."

However, since this is a statement about an activity they've been doing, it's not necessarily a factual info about themselves. A more neutral extraction would be:

"Has experience with Docker containers." 

But in the format requested, I'll keep the original: 

I recently started working with Docker containers.
Saved to Memory automatically.
